<a href="https://colab.research.google.com/github/ChickenRob0t/W_Scrapping_Meteorologia/blob/main/Ambient_Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta


import pandas as pd
import time

In [ ]:
#Código para eliminar muchos problemas con Selenium
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

#https://github.com/googlecolab/colabtools/issues/3347 Para recordar xd, sirve en mayo :p

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,084 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,372 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal 

In [ ]:
#Cosas que se necesitan desde un inicio


fecha_actual = datetime.now()                   #Problemas técnicos con la zona horaria para que me entregue el día de hoy siempre
desplazamiento_gdl = timedelta(hours=-6)

date_p= (fecha_actual + desplazamiento_gdl ).strftime('%Y-%m-%d')  #Esto entrega la fecha de hoy, no importa cuando lo corra.

tai='502a442a99dea9fae77d207ff08f85e1'

#'Tlacayapan':['0dff2b954d9e73ca3530a1f30afe4a97',20.302, 103.234], al parecer aquí ya no suben las tablas cx

Estaciones={'Zapopan1': ['b3e63eec573af419b148408a5a07c6fb',20.695, 103.419], 'AXLZAP': ['641d418cb6ad421cd1afef7cb77a032a',20.671, 103.428],'Juarez_Ton': ['86e65016ee64e3fe0f38a2a6459ce364',20.625, 103.242],
'Manantial':[ '502a442a99dea9fae77d207ff08f85e1', 20.621, 103.344], 'Tulios': ['2f00329e2440c1c8d88f1d2284c2faca', 20.605, 103.450], 'Rioja': ['1b9e6178b98a5d0201285190472c6431',20.570, 103.464],
'San_Agustin':['1a14068c0c420cca7e3892eb62227976',20.417, 103.389], 'Hyb_Juan_Oc':['db1c8f0e8e769eb44eb87141104b1f81',20.747, 103.512], 'Ajijic': ['304403337b6c294fa7cc21619a32effb',20.303, 103.273],
'Chapa_Rem':['51291e098b633ffc28b9cb916f56ec8f', 20.302, 103.185],  'Rancho_tenama':['3f3f302d2db34a73c58548664184cc29', 21.632, 102.002],
'San_José_Tepoz': ['706206e35560608a54ebbf45ebb05c21', 21.495, 102.177], 'Terra_Arandas':['8ca15b6e1eedab593aa46a6a10d1a209', 20.722, 102.340], 'Guamuchil_reparo':['cde1aa170622b561d6b7c24c1c8a1745', 19.911, 103.536],
'Meteo_CdGman':['a9eee3b5ed6dfe95d4d7fbb082aa940c', 19.686, 103.464], 'Willy_Manz':['370ce2ea66a5c3f19cc370b34beec80f', 19.284, 104.785], 'Puerto_V':['c308e37a0d821b9251363aac297babbd',20.641, 105.229],
'Rosario_cañ':['1d0db67505fbbb23c6c44b7b7d53ced4', 21.047, 102.586]}

In [ ]:
 #Función para pasar de Farenheit a Celsius
def To_celsius(x):
  if '°' in x:
    x=float(x.replace('°','').replace('F',''))
    return round((x-32)*5/9 , 1)
  else:
    return x

#Esto sirve para acomodar las bases de datos, al formato que queremos
def filt(Df):
  if "Today'S High/Low" in Df.index.values:
    Df=Df.loc[["Temperature", "Today'S High/Low"]].transpose()
    return Df
  else:
    Df=Df.loc[["Temperature"]].transpose()
    return Df


In [ ]:
errores=[]
def tabla_est(tail):

  #Le paso la URL a través del código que diferencia las estaciones
  URL="https://ambientweather.net/dashboard/{}".format(tail)

  options = webdriver.ChromeOptions()
  options.add_argument("--headless")
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument("--no-sandbox")

  #Obtengo la información de la URL con las opciones predeterminadas
  driver = webdriver.Chrome(options=options)
  driver.get(URL)

  #Encontrar la tabla en la página
  Table=driver.find_element(By.CSS_SELECTOR, "div.device-quick-view-widget.small-flex-lines")
  lis=Table.text.split(sep='\n')

  index=[]                                                 #Arreglamos algunos elementos que se pasaron mal
  lis=list(map(To_celsius, lis))                         #Pasamos de Farenheit a Celsius
  for i in range(len(lis)):
    if lis[i]=='/':
      lis[i]="de {}{}{}".format(lis[i-1],' a ',lis[i+1])
      index.append(i+1)
      index.append(i-1)



  for idx in sorted(index,reverse=True):
    del lis[idx]                                               #Eliminamos los que no sirven

  try:
    sol=lis.index('Solar')
    lis.pop(sol+2) ;                                              #Otros elementos que son de la parte solar que no nos sirven
    lis.pop(sol+1);
    lis.pop(sol);
  except ValueError:
    errores.append(tail)



  #Generamos la tabla de la página y la hacemos una base de datos

  impar=[element for idx, element in enumerate(lis) if idx %2 != 0 ]
  par= lis[::2]
  Tabla_df=pd.DataFrame({'Name': par, 'Valores': impar}).set_index('Name')


  return filt(Tabla_df)

In [ ]:
#Hacemos una lista con los dataframes de cada estacion
#El inicio y el fin de estas sólo es para ver si hay un problema, para ver cuál es, aveces dejan de poner las tablas
Tabla_completa=pd.DataFrame()
for i in Estaciones:
  print(i)
  aux=tabla_est(Estaciones[i][0])
  aux[['Estacion','Latitud','Longitud','Fecha']]=[i,Estaciones[i][1],Estaciones[i][2],date_p]

  Tabla_completa=pd.concat([Tabla_completa,aux])
  print('Fin')
Tabla_completa

Zapopan1
Fin
AXLZAP
Fin
Juarez_Ton
Fin
Manantial
Fin
Tulios
Fin
Rioja
Fin
San_Agustin
Fin
Hyb_Juan_Oc
Fin
Ajijic
Fin
Chapa_Rem
Fin
Rancho_tenama
Fin
San_José_Tepoz
Fin
Terra_Arandas
Fin
Guamuchil_reparo
Fin
Meteo_CdGman
Fin
Willy_Manz
Fin
Puerto_V
Fin
Rosario_cañ
Fin


Name,Temperature,Today'S High/Low,Estacion,Latitud,Longitud,Fecha
Valores,30.9,de 31.9 a 19.7,Zapopan1,20.695,103.419,2023-07-16
Valores,29.9,de 31.7 a 19.7,AXLZAP,20.671,103.428,2023-07-16
Valores,30.2,de 31.0 a 19.5,Juarez_Ton,20.625,103.242,2023-07-16
Valores,30.6,NaN,Manantial,20.621,103.344,2023-07-16
Valores,30.3,de 33.4 a 19.1,Tulios,20.605,103.450,2023-07-16
Valores,30.9,de 32.8 a 19.7,Rioja,20.570,103.464,2023-07-16
Valores,31.2,de 32.1 a 20.0,San_Agustin,20.417,103.389,2023-07-16
Valores,29.7,NaN,Hyb_Juan_Oc,20.747,103.512,2023-07-16
Valores,34.9,NaN,Ajijic,20.303,103.273,2023-07-16
Valores,30.3,NaN,Chapa_Rem,20.302,103.185,2023-07-16


In [ ]:
Tabla_completa.to_csv('{}_Ambient_weather.csv'.format(date_p))